In [1]:
from astropy.io import fits, ascii
from astropy.table import Table

from astropy import wcs
import os
import numpy as np
from ants import cvMe, fitsPlay
from scipy.ndimage.interpolation import zoom

#import sharpener
#from sharpener import mirlib
import shutil
from matplotlib import pyplot as plt
from matplotlib import rc
from matplotlib import gridspec
from matplotlib.ticker import AutoMinorLocator, MultipleLocator, LogLocator
from matplotlib import transforms as mtransforms
from matplotlib.ticker import LogFormatter 
from matplotlib.colors import LogNorm
from matplotlib.colors import SymLogNorm

conv = cvMe.convert()
fp = fitsPlay.fitsplay()


get_ipython().magic(u'pylab inline')

home = '/home/'
#home = '/Users/'

rootDir = home+'maccagni/Projects/MFS/FornaxA/MUSE/'
cubeDir = rootDir+'cubes/'

GAL = 'FornaxE'
RUN = '_4750-6800-v100'

resDir= cubeDir+'results/'+GAL+RUN+'/mycubes/'

resPlots = cubeDir+'results/'+GAL+RUN+'/myplots/'



print '''\t+---------+\n\t Inputs loaded\n\t+---------+''' 

Populating the interactive namespace from numpy and matplotlib
	+---------+
	 Inputs loaded
	+---------+


/home/maccagni/programs/anaconda/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['rc']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
#----------------------#
# rc param initialize
#----------------------#
params = {'figure.figsize'      : '10,10',
          'font.family'         :' serif',
          'font.serif'          :'times',
          'font.style'          : 'normal',
          'font.weight'         : 'book',
          'font.size'           : 24,
          'axes.linewidth'      : 2,
          'lines.linewidth'     : 3,
          'xtick.labelsize'     : 22,
          'ytick.labelsize'     : 22, 
          'xtick.direction'     :'in',
          'ytick.direction'     :'in',
          'xtick.major.size'    : 6,
          'xtick.major.width'   : 2,
          'xtick.minor.size'    : 3,
          'xtick.minor.width'   : 1,
          'ytick.major.size'    : 6,
          'ytick.major.width'   : 2,
          'ytick.minor.size'    : 3,
          'ytick.minor.width'   : 1, 
          'text.usetex'         : True,
          'text.latex.unicode'  : True
           }
plt.rcParams.update(params)

print '''\n\t+---------+\n\t Matplotlib initialized\n\t+---------+'''


	+---------+
	 Matplotlib initialized
	+---------+


In [3]:
#----------------------#
# Convert to VRAD
#----------------------#

oIIIVrad= resDir+'HbetaCubeVrad.fits'

f = fits.open(oIIIVrad)
hh = f[0].header
dd = f[0].data

vrotMin = 1760-250.
vrotMax = 1760+250.

vel = ((np.linspace(1, dd.shape[0], dd.shape[0]) - hh['CRPIX3']) * hh['CDELT3'] + hh['CRVAL3']) /1e3

indMin = (np.abs(vel - vrotMin)).argmin()
indMax = (np.abs(vel - vrotMax)).argmin()+1

print indMax

print '''\t+---------+\n\t fits loaded\n\t+---------+'''     

16
	+---------+
	 fits loaded
	+---------+


In [4]:
def write1GaussFitTable(dictFit,outTable):

    c = ['ID','BIN_ID','X','Y','PixX','PixY',
                  'centre','centreErr','amplitude','ampErr',
                  'sigma','sigmaErr','fwhm','height',
                  'redchi', 'bic', 'aic','success',
                  'ndata','nvarys','nfev']
    
    allData = np.array([dictFit['ID'], dictFit['BIN_ID'], np.round(dictFit['X'],1),np.round(dictFit['Y'],1), dictFit['PixX'], dictFit['PixY'],
              dictFit['center'], dictFit['centreErr'], dictFit['amplitude'], dictFit['ampErr'],
              dictFit['sigma'], dictFit['sigmaErr'], dictFit['fwhm'], dictFit['height'],
              dictFit['redchi'], dictFit['bic'], dictFit['aic'], dictFit['success'],
              dictFit['ndata'], dictFit['nvarys'], dictFit['nfev']])
    
    allData = allData.astype('str')

    tt = Table(allData,names=c,meta={'name': '1GaussianFit'})

    if os.path.exists(outTable):
        tt = Table.read(outTable, format='ascii')
        tt.add_row(allData)
    else: 
        tt = Table(allData,names=c,meta={'name': '1GaussianFit'})    

    ascii.write(tt,outTable, overwrite=True)
    return 0
        
def makePlt1Gauss(vel,y,yBFit,yRes,dely,outPlot):
            
     # initialize figure
    fig = plt.figure(figsize =(10,10))
    fig.subplots_adjust(hspace=0.0)
    gs = gridspec.GridSpec(1, 1)
    plt.rc('xtick')

    # Initialize subplots
    ax1 = fig.add_subplot(gs[0])
    ax1.set_xlabel(r'Velocity [km s$^{-1}$]')
    ax1.set_ylabel(r'Flux [-]')
    #ax1.set_xlim([])
    #ax1.set_ylim([])
    ax1.tick_params(axis='both', which='major', pad=5)
    #ax1.xaxis.set_minor_locator()
    #ax1.yaxis.set_minor_locator()      
    
    ax1.step(vel, y, where='mid', color='black', linestyle='-')
    ax1.plot(vel, yBFit, 'r-', label='best fit')
    ax1.step(vel, yRes, 'g-', label='residuals')
    ax1.fill_between(vel, yBFit-dely, yBFit+dely, color="#ABABAB",
            label='3-$\sigma$ uncertainty band')
    ax1.legend(loc='best')
    plt.savefig(outPlot,
                format='png') # if pdf,dpi=300,transparent=True,bbox_inches='tight',overwrite=True)
    plt.close()
       
    return 0

print '''\t+---------+\n\t modules loaded\n\t+---------+'''            

	+---------+
	 modules loaded
	+---------+


In [ ]:
from lmfit import Model
from lmfit.models import GaussianModel
from lmfit.model import save_modelresult


Gmod = GaussianModel()


cubeSNR = resDir+'HbetaCubeVradSNR.fits'
mom0Name = resDir+'Hbeta_mom0-1G.fits'
mom1Name = resDir+'Hbeta_mom1-1G.fits'

inTable = resDir+'../'+GAL+'_table.fits'

outTable = resDir+'../'+GAL+'_1GausFitTable.txt'

tab = fits.open(inTable)
head = tab[0].header
headTab = tab[1].header
dataTab = tab[1].data

crPix1=200 # E 
crPix2=125
head['CRPIX1'] = crPix1
head['CRPIX2'] = crPix2

ID = dataTab['ID']
print ID
BIN_ID = dataTab['BIN_ID']

Xcoord = dataTab['X']
Ycoord = dataTab['Y']

xMin = np.min(dataTab['X'])
xMax = np.max(dataTab['X'])

shapeX = (xMax-xMin)/head['PIXSIZE']

yMin = np.min(dataTab['Y'])
yMax = np.max(dataTab['Y'])

shapeY = (yMax-yMin)/head['PIXSIZE']

xAxis = (np.linspace(1, shapeX+1, shapeX+1) - head['CRPIX1']) *head['PIXSIZE']
yAxis = (np.linspace(1, shapeY+1, shapeY+1) - head['CRPIX2']) *head['PIXSIZE']


f = fits.open(cubeSNR)
hh = f[0].header
dd = f[0].data
#SNR
vel = ((np.linspace(1, dd.shape[0], dd.shape[0]) - hh['CRPIX3']) * hh['CDELT3'] + hh['CRVAL3']) /1e3 - 1760
#SNROT
#vel = ((np.linspace(1, dd.shape[0], dd.shape[0]) - hh['CRPIX3']) * hh['CDELT3'] + hh['CRVAL3']) /1e3 

diffusion = 1e-3

mom0 = np.zeros([dd.shape[1],dd.shape[2]])
mom1 = np.zeros([dd.shape[1],dd.shape[2]])

#create AllSpectra datacube
#for j in xrange(123,125):
#    for i in xrange(199,201):

for j in xrange(0,dd.shape[1]):
    for i in xrange(0,dd.shape[2]):
 
        y = dd[:,j,i]
           
        if np.sum(y)>0:

            xVal = xAxis[i]
            yVal = yAxis[j]

            index = np.where((Xcoord < (xVal+head['PIXSIZE']/2.)) & 
            ((xVal-head['PIXSIZE']/2.) < Xcoord) & (Ycoord < (yVal+head['PIXSIZE']/2.)) & 
            ((yVal-head['PIXSIZE']/2.) < Ycoord))


            binID = BIN_ID[index][0]
            IDName = ID[index][0]
            xx = Xcoord[index][0]
            yy = Ycoord[index][0]
            PixX = i
            PixY = j
            
            tabInfo = {'ID':IDName,'BIN_ID':binID, 'X':xx, 'Y':yy, 'PixX':PixX, 'PixY': PixY}
            
            pars = Gmod.guess(y, x=vel)
            result = Gmod.fit(y, pars, x=vel)
            fitRes = result.params.valuesdict()
            errors = result.conf_interval()

            
            dely = result.eval_uncertainty(sigma=3)
            yBFit = result.best_fit
            yRes = result.residual
            outPlot = resPlots+'Hbeta/'+str(i)+'_'+str(j)+'_snr'       
            
            makePlt1Gauss(vel, y,yBFit,yRes,dely,outPlot)
            
            mom0[j,i] = fitRes['amplitude']
            mom1[j,i] = fitRes['center']
     
            amp = fitRes['amplitude']
            ctr = fitRes['center']
            sig = fitRes['sigma']
            
            fwhm = fitRes['fwhm']
            height = fitRes['height']
            
            
            amp_err = result.params['amplitude'].stderr
            sig_err = result.params['sigma'].stderr
            cen_err = result.params['center'].stderr            
                    
            tabErr = {'amplitude':amp, 'center': ctr, 'sigma': sig, 'fwhm' : fwhm, 'height' : height,
                      'centreErr':cen_err,'ampErr': amp_err, 'sigmaErr': sig_err}
            
            aic = result.aic
            bic = result.bic
            redchi = result.redchi
            success = result.success
            ndata = result.ndata
            nvarys = result.nvarys
            nfev = result.nfev

            tabFit = {'aic':aic,'bic': bic, 'ndata': ndata, 
                      'redchi': redchi, 'success': success,
                     'ndata' : ndata, 'nvarys' : nvarys,
                     'nfev': nfev}
            
            tabInfo.update(tabErr)
            tabInfo.update(tabFit)
            
            write1GaussFitTable(tabInfo,outTable)
            
        
            
        else:
            pass
        
        
mom1[mom1==0.0] = np.nan    
mom0[mom0==0.0] = np.nan    

    
fits.writeto(mom0Name,mom0,hh,overwrite=True)
fits.writeto(mom1Name,mom1,hh,overwrite=True)        
        
print '''\t+---------+\n\t fits saved\n\t+---------+'''     

[    0     1     2 ... 96159 96160 96161]


/home/maccagni/programs/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:44: DeprecationWarning: object of type <type 'numpy.float64'> cannot be safely interpreted as an integer.
/home/maccagni/programs/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:45: DeprecationWarning: object of type <type 'numpy.float64'> cannot be safely interpreted as an integer.
/home/maccagni/programs/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:1328: UserWarning: findfont: Font family [u'serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))
/home/maccagni/programs/anaconda/lib/python2.7/site-packages/lmfit/confidence.py:318: UserWarning: rel_change=9.83596017887e-06 < 1e-05 at iteration 76 and prob(sigma=4031.0145798) = 0.96598798639 < max(sigmas).
  warn(errmsg)
/home/maccagni/programs/anaconda/lib/python2.7/site-packages/lmfit/confidence.py:310: UserWarning: maxiter=200 reached and prob(amplitude=6969296.69736) = 0.9557140

In [ ]:
#----------------------#
# Filter Cubes
#----------------------#

mom0Name = resDir+'OIII_mom0Rot-NoFit.fits'
mom1Name = resDir+'OIII_mom1Rot-NoFit.fits'

cubeSNRRot = resDir+'OIIICubeVradSNRRot.fits'
cubeSNRBroad = resDir+'OIIICubeVradSNRBroad.fits'
noiseName = resDir+'OIIICubeNoise.fits'


SNR= 3.
SNRVel= 5.

#measure noise along spaxel in empty region
f = fits.open(oIIIVrad)
hh = f[0].header
dd = f[0].data

indexCubeA = 0
indexCubeB = 24
indexCubeC = 50
indexCubeD = 66

lineCube = resDir+'LineCube.fits'
f = fits.open(lineCube)
lines = f[0].data

newDD = dd.copy()*0.0
newDD = np.multiply(newDD,0.0)

newBB = dd.copy()*0.0
newBB = np.multiply(newBB,0.0)

mom0 = np.zeros([newDD.shape[1],newDD.shape[2]])
mom1 = np.zeros([newDD.shape[1],newDD.shape[2]])


noiseCube = dd.copy()*0.0
noiseCube = np.multiply(noiseCube,0.0)

#create AllSpectra datacube
for j in xrange(0,dd.shape[1]):
    for i in xrange(0,dd.shape[2]):    
        
        noiseA = np.std(lines[indexCubeA:indexCubeB,j,i])
        noiseB = np.std(lines[indexCubeC:indexCubeD,j,i])
        noise = np.mean([noiseA,noiseB])
        
        maximum = np.max(dd[indexCubeB:indexCubeC,j,i])
        SNRLim = SNR*noise 
        SNRLimVel = SNRVel*noise 
              
        if maximum > SNRLim: 

            arr = dd[indMin:indMax,j,i] 

            arr[arr<=SNRLim] = 0.0
            newDD[indMin:indMax,j,i] = arr         
            
            barr = dd[:,j,i]
            barr[indMin:indMax] = 0
            newBB[:,j,i] = barr
     
            mom0[j,i] = np.sum(arr)
            noiseCube[:,j,i] = SNRLim
            
        else:
            newDD[:,j,i] = np.nan

        if maximum > SNRLimVel:   
            
            arr = dd[indMin:indMax,j,i]
            
            arr[arr<=SNRLimVel] = 0.0
            
            #peak mom1
        #    mom1[j,i] = np.divide(np.sum(np.multiply(np.max(arr[indexCubeB:indexCubeC]),vel[np.argmax(arr[indexCubeB:indexCubeC])]))
        #                          ,np.sum(arr[indexCubeB:indexCubeC]))
            
            mom1[j,i] = np.divide(np.sum(np.multiply(arr,vel[indMin:indMax])),np.sum(arr))

            
hh['RESTFREQ'] = 5.987741757660128e13
hh['SPECSYS'] = 'BARYCENT'
hh['CRVAL3'] =  hh['CRVAL3']-1760e3

print hh['CRVAL3']
fits.writeto(cubeSNRRot,newDD,hh,overwrite=True)

fits.writeto(cubeSNRBroad,newBB,hh,overwrite=True)

fits.writeto(noiseName,noiseCube,hh,overwrite=True)

if 'NAXIS3' in hh:
    del hh['NAXIS3']
if 'CRVAL3' in hh:
    hh['FREQ'] = hh['CRVAL3']
    del hh['CRVAL3']
if 'CDELT3' in hh:
    del hh['CDELT3']
if 'CRPIX3' in hh: 
    del hh['CRPIX3']
if 'CTYPE3' in hh:        
    del hh['CTYPE3']  
if 'CROTA3' in hh:
    del hh['CROTA3']
if 'CUNIT3' in hh:
    del hh['CUNIT3'] 
    
    
mom1[mom1==0.0] = np.nan    
mom0[mom0==0.0] = np.nan    

    
fits.writeto(mom0Name,mom0,hh,overwrite=True)
fits.writeto(mom1Name,mom1,hh,overwrite=True)


print '''\t+---------+\n\t fits saved\n\t+---------+'''     

In [ ]:
#----------------------#
# Convert to VRAD
#----------------------#

oIIIVrad= resDir+'HbetaCubeVrad.fits'

f = fits.open(oIIIVrad)
hh = f[0].header
dd = f[0].data

vrotMin = 1760-250.
vrotMax = 1760+250.

vel = ((np.linspace(1, dd.shape[0], dd.shape[0]) - hh['CRPIX3']) * hh['CDELT3'] + hh['CRVAL3']) /1e3 

indMin = (np.abs(vel - vrotMin)).argmin()
indMax = (np.abs(vel - vrotMax)).argmin()+1

print indMax

print '''\t+---------+\n\t fits loaded\n\t+---------+'''     

In [130]:
#----------------------#
# Filter Cubes
#----------------------#

mom0Name = resDir+'Hbeta_mom0Rot-NoFit.fits'
mom1Name = resDir+'Hbeta_mom1Rot-NoFit.fits'

cubeSNRRot = resDir+'HbetaCubeVradSNRRot.fits'

cubeSNRBroad = resDir+'HbetaCubeVradSNRBroad.fits'
noiseName = resDir+'HbetaCubeNoise.fits'


SNR= 3.
SNRVel= 5.

#measure noise along spaxel in empty region
f = fits.open(oIIIVrad)
hh = f[0].header
dd = f[0].data

indexCubeA = 0
indexCubeB = 2
indexCubeC = 17
indexCubeD = 22

lineCube = resDir+'LineCube.fits'
f = fits.open(lineCube)
lines = f[0].data

newDD = dd.copy()*0.0
newDD = np.multiply(newDD,0.0)

newBB = dd.copy()*0.0
newBB = np.multiply(newBB,0.0)

mom0 = np.zeros([newDD.shape[1],newDD.shape[2]])
mom1 = np.zeros([newDD.shape[1],newDD.shape[2]])


noiseCube = dd.copy()*0.0
noiseCube = np.multiply(noiseCube,0.0)

#create AllSpectra datacube
for j in xrange(0,dd.shape[1]):
    for i in xrange(0,dd.shape[2]):    
        
        noiseA = np.std(lines[indexCubeA:indexCubeB,j,i])
        noiseB = np.std(lines[indexCubeC:indexCubeD,j,i])
        noise = np.mean([noiseA,noiseB])
        
        maximum = np.max(dd[indexCubeB:indexCubeC,j,i])
        SNRLim = SNR*noise 
        SNRLimVel = SNRVel*noise 
              
        if maximum > SNRLim: 

            arr = dd[indMin:indMax,j,i] 

            arr[arr<=SNRLim] = 0.0
            newDD[indMin:indMax,j,i] = arr         
            
            barr = dd[:,j,i]
            barr[indMin:indMax] = 0
            newBB[:,j,i] = barr
     
            mom0[j,i] = np.sum(arr)
            noiseCube[:,j,i] = SNRLim
            
        else:
            newDD[:,j,i] = np.nan

        if maximum > SNRLimVel:   
            
            arr = dd[indMin:indMax,j,i]
            
            arr[arr<=SNRLimVel] = 0.0
            
            #peak mom1
        #    mom1[j,i] = np.divide(np.sum(np.multiply(np.max(arr[indexCubeB:indexCubeC]),vel[np.argmax(arr[indexCubeB:indexCubeC])]))
        #                          ,np.sum(arr[indexCubeB:indexCubeC]))
            
            mom1[j,i] = np.divide(np.sum(np.multiply(arr,vel[indMin:indMax])),np.sum(arr))

            
hh['RESTFREQ'] = 5.987741757660128e13
hh['SPECSYS'] = 'BARYCENT'
hh['CRVAL3'] =  hh['CRVAL3']-1760e3

print hh['CRVAL3']
fits.writeto(cubeSNRRot,newDD,hh,overwrite=True)

fits.writeto(cubeSNRBroad,newBB,hh,overwrite=True)

fits.writeto(noiseName,noiseCube,hh,overwrite=True)

if 'NAXIS3' in hh:
    del hh['NAXIS3']
if 'CRVAL3' in hh:
    hh['FREQ'] = hh['CRVAL3']
    del hh['CRVAL3']
if 'CDELT3' in hh:
    del hh['CDELT3']
if 'CRPIX3' in hh: 
    del hh['CRPIX3']
if 'CTYPE3' in hh:        
    del hh['CTYPE3']  
if 'CROTA3' in hh:
    del hh['CROTA3']
if 'CUNIT3' in hh:
    del hh['CUNIT3'] 
    
    
mom1[mom1==0.0] = np.nan    
mom0[mom0==0.0] = np.nan    

    
fits.writeto(mom0Name,mom0,hh,overwrite=True)
fits.writeto(mom1Name,mom1,hh,overwrite=True)



print '''\t+---------+\n\t fits saved\n\t+---------+'''     

/home/maccagni/programs/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:83: RuntimeWarning: invalid value encountered in divide


-698092.416167
	+---------+
	 fits saved
	+---------+


In [15]:
#----------------------#
# Convert to VRAD
#----------------------#

oIIIVrad= resDir+'OICubeVrad.fits'

f = fits.open(oIIIVrad)
hh = f[0].header
dd = f[0].data

vrotMin = 1760-250.
vrotMax = 1760+250.

vel = ((np.linspace(1, dd.shape[0], dd.shape[0]) - hh['CRPIX3']) * hh['CDELT3'] + hh['CRVAL3']) /1e3 

indMin = (np.abs(vel - vrotMin)).argmin()
indMax = (np.abs(vel - vrotMax)).argmin()+1

print indMax

print '''\t+---------+\n\t fits loaded\n\t+---------+'''     

16
	+---------+
	 fits loaded
	+---------+


In [16]:
#----------------------#
# Filter Cubes
#----------------------#

cubeSNR = resDir+'OICubeVradSNRRot.fits'
mom0Name = resDir+'OI_mom0Rot.fits'
mom1Name = resDir+'OI_mom1Rot.fits'

cubeSNR = resDir+'OICubeVradSNRRot.fits'
cubeSNRBroad = resDir+'OICubeVradSNRBroad.fits'
noiseName = resDir+'OICubeNoise.fits'


SNR= 3.
SNRVel= 5.

#measure noise along spaxel in empty region
f = fits.open(oIIIVrad)
hh = f[0].header
dd = f[0].data

indexCubeA = 0
indexCubeB = 2
indexCubeC = 25
indexCubeD = 61

lineCube = resDir+'LineCube.fits'
f = fits.open(lineCube)
lines = f[0].data

newDD = dd.copy()*0.0
newDD = np.multiply(newDD,0.0)

newBB = dd.copy()*0.0
newBB = np.multiply(newBB,0.0)

mom0 = np.zeros([newDD.shape[1],newDD.shape[2]])
mom1 = np.zeros([newDD.shape[1],newDD.shape[2]])


noiseCube = dd.copy()*0.0
noiseCube = np.multiply(noiseCube,0.0)

#create AllSpectra datacube
for j in xrange(0,dd.shape[1]):
    for i in xrange(0,dd.shape[2]):    
        
        noiseA = np.std(lines[indexCubeA:indexCubeB,j,i])
        noiseB = np.std(lines[indexCubeC:indexCubeD,j,i])
        noise = np.mean([noiseA,noiseB])
        
        maximum = np.max(dd[indexCubeB:indexCubeC,j,i])
        SNRLim = SNR*noise 
        SNRLimVel = SNRVel*noise 
              
        if maximum > SNRLim: 

            arr = dd[indMin:indMax,j,i] 

            arr[arr<=SNRLim] = 0.0
            newDD[indMin:indMax,j,i] = arr         
            
            barr = dd[:,j,i]
            barr[indMin:indMax] = 0
            newBB[:,j,i] = barr
     
            mom0[j,i] = np.sum(arr)
            noiseCube[:,j,i] = SNRLim
            
        else:
            newDD[:,j,i] = np.nan

        if maximum > SNRLimVel:   
            
            arr = dd[indMin:indMax,j,i]
            
            arr[arr<=SNRLimVel] = 0.0
            
            #peak mom1
        #    mom1[j,i] = np.divide(np.sum(np.multiply(np.max(arr[indexCubeB:indexCubeC]),vel[np.argmax(arr[indexCubeB:indexCubeC])]))
        #                          ,np.sum(arr[indexCubeB:indexCubeC]))
            
            mom1[j,i] = np.divide(np.sum(np.multiply(arr,vel[indMin:indMax])),np.sum(arr))

            
hh['RESTFREQ'] = 5.987741757660128e13
hh['SPECSYS'] = 'BARYCENT'
hh['CRVAL3'] =  hh['CRVAL3']-1760e3

print hh['CRVAL3']
fits.writeto(cubeSNR,newDD,hh,overwrite=True)

fits.writeto(cubeSNRBroad,newBB,hh,overwrite=True)

fits.writeto(noiseName,noiseCube,hh,overwrite=True)

if 'NAXIS3' in hh:
    del hh['NAXIS3']
if 'CRVAL3' in hh:
    hh['FREQ'] = hh['CRVAL3']
    del hh['CRVAL3']
if 'CDELT3' in hh:
    del hh['CDELT3']
if 'CRPIX3' in hh: 
    del hh['CRPIX3']
if 'CTYPE3' in hh:        
    del hh['CTYPE3']  
if 'CROTA3' in hh:
    del hh['CROTA3']
if 'CUNIT3' in hh:
    del hh['CUNIT3'] 
    
    
mom1[mom1==0.0] = np.nan    
mom0[mom0==0.0] = np.nan    

    
fits.writeto(mom0Name,mom0,hh,overwrite=True)
fits.writeto(mom1Name,mom1,hh,overwrite=True)


print '''\t+---------+\n\t fits saved\n\t+---------+'''     

/Users/maccagni/programs/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:83: RuntimeWarning: invalid value encountered in divide


-723285.826101
	+---------+
	 fits saved
	+---------+
